# Классификация текстов

В этом ноутбуке мы разберем задачу классификации текстов на примере соревнования по выявлению токсичных твиттов: https://www.kaggle.com/competitions/toxic-comments-classification-2

Наша задача - построить классификатор, который по тексту твитта определяет, токсичный он или нет.

__План:__
1. Разбираем/освежаем в памяти простые бейзлайны: мешок слов, TF-IDF. 
2. Обучаем классификатор на основе w2v эмбеддингов
3. Знакомимся с моделью fastText

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import *

Обучающая выборка и тестовые данные для предикта - их можно скачать с Kaggle.

In [ ]:
train = pd.read_csv('train_data.csv')
test = pd.read_csv('test_data.csv')

In [ ]:
train.sample(3)

,comment,toxic
6506,Белоруская операция Багратион проводилась с 23...,0.0
5303,А какое смешение чувств отразилось на ебженово...,1.0
3332,Ты в маём серце казёл.\n,0.0


In [ ]:
test.sample()

,comment_id,comment
947,947,Неделю назад держал эту оправу в руках - посме...


In [ ]:
#y_test = pd.read_csv('test_labels.csv')

В моем распоряжении также есть файл с тестовыми метками классов - если Вы запускаете ноутбук и хотите обучить модель, разбейте train выборку на train и test, раскомментировав строчки кода в ячейке ниже:

In [ ]:
# train, test = train_test_split(x_train, test_size=0.3)
# y_test = test[['comment_id', 'toxic']]

# test.drop(columns=['toxic'], inplace=True)

In [ ]:
y_test.sample()

,comment_id,toxic
2570,2570,0


In [ ]:
y_test = y_test['toxic'].values
y_test

array([0, 0, 0, ..., 1, 0, 0])

### Мы начнем с простых бейзлайнов

Это всегда хорошая практика - сперва попробовать что-то предельно простое (: В нашем случае это будет логистическая регрессия + мешок слов (Bag of Words, BoW).

In [ ]:
from sklearn.linear_model import LogisticRegression 
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vec = CountVectorizer(ngram_range=(1, 1), token_pattern='\w{3,}') # строим BoW для слов

In [ ]:
#help(CountVectorizer)

In [ ]:
bow = vec.fit_transform(train['comment'])

In [ ]:
bow

<10809x55942 sparse matrix of type '<class 'numpy.int64'>'
	with 210108 stored elements in Compressed Sparse Row format>

In [ ]:
print(train.comment[10804])

А у мамы в группе до самого выпуска из сада такие просяки нет-нет, да случаются, с разными детьми. Только для одних детей это разовая акция, у других - время от времени, а у третьих - частенько. И родители прекрасно в курсе особенностей каждого конкретного ребенка.



In [ ]:
list(vec.vocabulary_.items())[:10]

[('преступление', 37067),
 ('наказание', 24836),
 ('именно', 16509),
 ('эти', 55542),
 ('неработающие', 26582),
 ('весы', 5914),
 ('показывают', 34521),
 ('что', 54262),
 ('работающих', 39917),
 ('нет', 26702)]

In [ ]:
sorted(list(vec.vocabulary_.items()), key=lambda x: x[0])[:10]

[('000', 0),
 ('0015', 1),
 ('003', 2),
 ('0036', 3),
 ('005', 4),
 ('00х', 5),
 ('013', 6),
 ('030050', 7),
 ('0611', 8),
 ('068', 9)]

In [ ]:
list(vec.vocabulary_.keys())[:10]

['преступление',
 'наказание',
 'именно',
 'эти',
 'неработающие',
 'весы',
 'показывают',
 'что',
 'работающих',
 'нет']

In [ ]:
len(vec.vocabulary_.items())

55942

In [ ]:
y_train = train['toxic'].astype(int).values
y_train

array([0, 0, 0, ..., 1, 0, 0])

In [ ]:
#help(clf.fit)

In [ ]:
clf = LogisticRegression(random_state=42, max_iter=500) #
clf.fit(bow, y_train)

LogisticRegression(max_iter=500, random_state=42)

In [ ]:
len(clf.coef_[0])

55942

In [ ]:
bow_test = vec.transform(test['comment'])
bow_test

<3603x55942 sparse matrix of type '<class 'numpy.int64'>'
	with 59233 stored elements in Compressed Sparse Row format>

In [ ]:
pred = clf.predict(bow_test)
pred[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0])

In [ ]:
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

           0       0.94      0.84      0.89      2651
           1       0.66      0.84      0.74       952

    accuracy                           0.84      3603
   macro avg       0.80      0.84      0.81      3603
weighted avg       0.86      0.84      0.85      3603



### Попробуем добавить препроцессинг текста

Препроцессинг, как правило, включает удаление небуквенных символов, топ-слов и нормализацию (стемминг - приведение к основе слова - или лемматизацию - приведение слов к начальной форме).

Кроме того, заменим мешок слов на TF-IDF матрицу. В качестве модели оставим логистическую регрессию.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
#!pip install pymorphy2

In [ ]:
#import nltk
#nltk.download('stopwords')

Функция для удаления небуквенных символов из текста:

In [ ]:
import re
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords

from functools import lru_cache
from tqdm.notebook import tqdm

m = MorphAnalyzer()
regex = re.compile("[а-яa-zёЁ]+")

def words_only(text, regex=regex):
    try:
        return regex.findall(text.lower())
    except:
        return []

In [ ]:
train.comment[1]

'И именно эти неработающие весы показывают, что работающих нет?..\n'

In [ ]:
words_only(train.comment[1])

['и',
 'именно',
 'эти',
 'неработающие',
 'весы',
 'показывают',
 'что',
 'работающих',
 'нет']

Функции для препроцессинга текста: 

1. Удаление небуквенных символов
2. Лемматизация 
3. Удаление коротких (менее 3 символов) и стоп-слов

In [ ]:
@lru_cache(maxsize=128)
def lemmatize_word(token, pymorphy=m):
    return pymorphy.parse(token)[0].normal_form

def lemmatize_text(text):
    return [lemmatize_word(w) for w in text]


mystopwords = stopwords.words('russian') 
def remove_stopwords(lemmas, stopwords = mystopwords):
    return [w for w in lemmas if not w in stopwords and len(w) > 3]

def clean_text(text):
    tokens = words_only(text)
    lemmas = lemmatize_text(tokens)
    
    return ' '.join(remove_stopwords(lemmas))

In [ ]:
%time lemmatize_word('неработающие')

CPU times: user 476 µs, sys: 1 µs, total: 477 µs
Wall time: 483 µs


'неработающий'

In [ ]:
train.comment[1]

'И именно эти неработающие весы показывают, что работающих нет?..\n'

In [ ]:
clean_text(train.comment[1])

'именно неработающий весы показывать работать'

Проводим препроцессинг для train и test выборок:

In [ ]:
lemmas = list(tqdm(map(clean_text, train['comment']), total=len(train)))
    
train['lemmas'] = lemmas
train.sample(5)

  0%|          | 0/10809 [00:00<?, ?it/s]

,comment,toxic,lemmas
2743,забанили в дойта трете за ету картинку.... иди...,0.0,забанить дойта тереть картинка идти туда запос...
9640,"3 января возможно, на жд например так даты исп...",0.0,январь возможно например дата использоваться
7937,"Вот и гляньте все что вокруг дна это пустота, ...",0.0,глянуть вокруг пустота использоваться полка пр...
8737,"Сижу, любуюсь шрамом диаметром 10мм на руке. Ш...",0.0,сидеть любоваться шрам диаметр рука шрам украш...
6173,Один кондей дешевле чем каналка. Мощности толь...,0.0,кондей дешёвый каналка мощность номер весь зда...


In [ ]:
lemmas_test = list(tqdm(map(clean_text, test['comment']), total=len(test)))
    
test['lemmas'] = lemmas_test

  0%|          | 0/3603 [00:00<?, ?it/s]

Считаем TF-IDF матрицу и обучаем модель:

In [ ]:
vec = TfidfVectorizer(ngram_range=(1, 2)) # строим BoW для слов
tfidf = vec.fit_transform(train['lemmas'])

clf = LogisticRegression(random_state=42, max_iter=500)
clf.fit(tfidf, y_train)

pred = clf.predict(vec.transform(test['lemmas']))
accuracy_score(pred, y_test)

0.8245906189286706

In [ ]:
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

           0       0.98      0.80      0.88      2925
           1       0.52      0.93      0.67       678

    accuracy                           0.82      3603
   macro avg       0.75      0.87      0.77      3603
weighted avg       0.89      0.82      0.84      3603



## Word2Vec

Попробуем использовать эмбеддинги слов - для этого сперва обучим модель Word2Vec c помощью библиотеки gensim.

In [ ]:
from gensim.models import word2vec

In [ ]:
train.sample()

,comment,toxic,lemmas
2580,"Во имя оболганного Отца, гонимого Сына и опаль...",1.0,оболгать отец гонимый опальный здравый смысл о...


In [ ]:
#help( word2vec.Word2Vec)

In [ ]:
tokenized_tweets = [tweet.split() for tweet in train['lemmas'].values]

%time w2v = word2vec.Word2Vec(tokenized_tweets, workers=4, vector_size=200, min_count=10, window=3, sample=1e-3)

CPU times: user 997 ms, sys: 6.05 ms, total: 1 s
Wall time: 356 ms


In [ ]:
w2v.wv.most_similar(positive=['плюс'], topn=10)

[('результат', 0.999779224395752),
 ('процент', 0.9997552037239075),
 ('использовать', 0.9997541308403015),
 ('интернет', 0.9997532367706299),
 ('российский', 0.9997491240501404),
 ('пройти', 0.9997482299804688),
 ('особенно', 0.9997453093528748),
 ('родитель', 0.9997445940971375),
 ('итог', 0.9997416734695435),
 ('ремонт', 0.9997414946556091)]

Теперь у нас есть эмбеддинги для слов. Но как получить эмбеддинги для твитов?

Можно просто усреднить эмбеддинги слов, входящих в твит.

In [ ]:
def get_tweet_embedding(lemmas, model=w2v.wv, embedding_size=200):
    
    res = np.zeros(embedding_size)
    cnt = 0
    for word in lemmas.split():
        if word in model:
            res += np.array(model[word])
            cnt += 1
    if cnt:
        res = res / cnt
    return res

In [ ]:
get_tweet_embedding('привет тебе')

array([ 0.02949993,  0.00711666,  0.06066887,  0.03044587,  0.10291898,
       -0.07171509, -0.04426383,  0.17461261, -0.06926171,  0.10137955,
       -0.09722775, -0.10701384,  0.02675423,  0.04505605, -0.03336318,
       -0.06858105, -0.09589617, -0.01504105,  0.08262346, -0.16852067,
        0.09995066, -0.112864  , -0.00117192,  0.03875624,  0.02932889,
       -0.07457103, -0.0108106 , -0.08021963, -0.16292173,  0.05479762,
        0.09130507,  0.04518832,  0.01549321, -0.01175892,  0.05495853,
        0.05215223,  0.11390019,  0.00301457,  0.00722505, -0.12864642,
       -0.08600076, -0.00843721,  0.01221396,  0.09540205,  0.14712787,
       -0.02386563, -0.06534632, -0.00829514,  0.07441702,  0.04880077,
        0.05805714, -0.01788091, -0.06644073, -0.07768032, -0.00920226,
       -0.04443413,  0.03236924, -0.12367389, -0.10502499, -0.02312545,
       -0.03215751,  0.01574282, -0.0707434 ,  0.04139043, -0.10889189,
        0.04811262,  0.00269111,  0.1813492 , -0.10731281,  0.10

Для каждого твита из обучающей и тестовой выборки вычислим такой эмбеддинг:

In [ ]:
train['w2v_embedding'] = train['lemmas'].map(get_tweet_embedding)
test['w2v_embedding'] = test['lemmas'].map(get_tweet_embedding)

In [ ]:
clf = LogisticRegression(random_state=42, max_iter=500)
clf.fit(list(train['w2v_embedding'].values), y_train)

pred = clf.predict(list(test['w2v_embedding'].values))
accuracy_score(pred, y_test)

0.6697196780460727

In [ ]:
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

           0       0.98      0.67      0.80      3467
           1       0.07      0.60      0.12       136

    accuracy                           0.67      3603
   macro avg       0.52      0.64      0.46      3603
weighted avg       0.94      0.67      0.77      3603



## FastText

FastText - это модификация модели word2vec.

FastText использует не только векторы слов, но и векторы n-грам. В корпусе каждое слово автоматически представляется в виде набора символьных n-грамм. Скажем, если мы установим n=3, то вектор для слова "where" будет представлен суммой векторов следующих триграм: "<wh", "whe", "her", "ere", "re>" (где "<" и ">" символы, обозначающие начало и конец слова). Благодаря этому мы можем также получать вектора для слов, отсутствуюших в словаре, а также эффективно работать с текстами, содержащими ошибки и опечатки.

* [Статья](https://aclweb.org/anthology/Q17-1010)
* [Сайт](https://fasttext.cc/)
* [Руководство](https://fasttext.cc/docs/en/support.html)
* [Репозиторий](https://github.com/facebookresearch/fasttext)

Есть библиотека `fasttext` для питона (с готовыми моделями можно работать и через `gensim`).

На сайте проекта можно найти предобученные модели для 157 языков (в том числе русского): https://fasttext.cc/docs/en/crawl-vectors.html

Для начала, попробуем взять предобученную модель fastText с сайта проекта и заменить эмбеддинги в модели выше на эмбеддинги fastText.

Бонус: попробуйте взять модель с сайта проекта Rusvetores: https://rusvectores.org/ru/models/

In [ ]:
#!pip install fasttext==0.6.0

Found existing installation: fasttext 0.9.2
Uninstalling fasttext-0.9.2:
  Successfully uninstalled fasttext-0.9.2
  Using cached fasttext-0.6.0-cp39-cp39-macosx_10_9_x86_64.whl


In [ ]:
import fasttext
import fasttext.util

In [ ]:
help(fasttext.util.download_model)

Help on function download_model in module fasttext.util.util:

download_model(lang_id, if_exists='strict', dimension=None)
    Download pre-trained common-crawl vectors from fastText's website
    https://fasttext.cc/docs/en/crawl-vectors.html



In [ ]:
#fasttext.util.download_model('ru', if_exists='ignore')
ft = fasttext.load_model('cc.ru.300.bin')

In [ ]:
ft['привет']

array([ 0.06434693, -0.01527086, -0.06963537, -0.03582602,  0.01471584,
       -0.03503159,  0.02701715,  0.04161827, -0.00033126,  0.00355259,
        0.06979205,  0.06205348,  0.05154078,  0.03831509, -0.02394784,
       -0.03954181, -0.00189653, -0.11174394, -0.0407712 ,  0.09289949,
       -0.07412342, -0.05209147,  0.02017231,  0.04837443,  0.02212641,
        0.00856511, -0.03055364,  0.04733564,  0.04380886,  0.03856769,
        0.03442968,  0.05576854,  0.01513439,  0.14055566,  0.03365337,
       -0.02920472, -0.10305687, -0.09332671,  0.03085899, -0.11067575,
       -0.08992791,  0.05850704, -0.017424  ,  0.00120653, -0.07153153,
        0.10312843, -0.08066262, -0.00642456,  0.04408539, -0.05728461,
       -0.0179531 ,  0.03936698,  0.04778077, -0.04907751, -0.00909553,
        0.05588715, -0.00236535,  0.04878682, -0.01769035,  0.03295048,
        0.00906604,  0.08772802,  0.02970458, -0.04903899, -0.03025401,
       -0.04151824,  0.04931813, -0.02804473,  0.05716789,  0.03

In [ ]:
x = 'привет всем слушателям курса'
get_tweet_embedding(x, model=ft, embedding_size=300)

array([ 2.84749218e-02,  1.14055865e-02, -1.54750008e-02,  6.10717852e-03,
       -5.42343501e-03,  2.83443742e-03,  2.40256451e-03,  1.29073053e-02,
        3.05031866e-02, -1.99234379e-02,  6.13203850e-02,  4.42768331e-02,
        2.71531800e-02, -1.02064133e-02,  9.22483567e-04,  2.50384058e-02,
       -1.25383004e-02, -4.89095808e-02, -3.07890818e-02,  1.01918663e-01,
       -2.85800546e-02, -1.05811988e-01, -1.28629373e-02,  2.95597422e-02,
        2.13206490e-03,  1.26906892e-02, -2.97227059e-02,  2.77029723e-02,
       -1.21254625e-02, -4.76178443e-02, -6.68591424e-03,  3.05985650e-02,
        3.59081652e-02,  1.02970391e-01,  3.62780495e-02, -5.56655712e-02,
       -1.11200343e-01, -1.16946280e-01,  4.69890856e-02, -5.79430675e-02,
       -4.56299540e-03, -2.32621958e-03, -2.30524363e-03,  1.96370891e-02,
       -1.68996924e-02,  4.77626729e-02, -7.71877861e-02,  2.95996453e-02,
        3.40769021e-02, -3.43663241e-02,  5.55797149e-02,  1.05126291e-02,
        9.77615127e-03,  

In [ ]:
train['ft_embedding'] = train['lemmas'].apply(lambda x: get_tweet_embedding(x, model=ft, embedding_size=300))
print('train done')

test['ft_embedding'] = test['lemmas'].apply(lambda x: get_tweet_embedding(x, model=ft, embedding_size=300))

train done


In [ ]:
clf = LogisticRegression(random_state=42, max_iter=500)
clf.fit(list(train['ft_embedding'].values), y_train)

pred = clf.predict(list(test['ft_embedding'].values))
accuracy_score(pred, y_test)

0.8709408825978351

In [ ]:
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

           0       0.95      0.87      0.91      2618
           1       0.71      0.88      0.79       985

    accuracy                           0.87      3603
   macro avg       0.83      0.87      0.85      3603
weighted avg       0.89      0.87      0.87      3603



### fastText как классификатор

fastText также можно использовать в режиме классификатора:

In [ ]:
train.sample()

,comment,toxic,lemmas,w2v_embedding,ft_embedding
5404,"да ну, кем считается то? у нас, швабов, пожест...",1.0,считаться шваб жёсткий лично думать,"[0.05510629341006279, 0.010897617496084422, 0....","[0.04708024859428406, 0.0034161420539021493, 0..."


In [ ]:
with open('train_ft.txt', 'w') as f:
    for label, lemmas in list(zip(
        train['toxic'], train['lemmas']
    )):
        f.write(f"__label__{int(label)} {lemmas}\n")
        #print(f"__label__{int(label)} {lemmas}")

with open('test_ft.txt', 'w') as f:
    for label, lemmas in list(zip(
        train['toxic'], train['lemmas']
    )):
        f.write(f"__label__{int(label)} {lemmas}\n")

In [ ]:
!tail train_ft.txt

__label__0 операция спирт тампон скальпель доктор выбрасывать пригодиться
__label__0 думать уместить знание мочь дать любой родитель домашний условие
__label__1 татарин народец гнилой весьма весьма
__label__0 красиво постановка театральный рыжий китайский очень доставить
__label__0 печально близкий нормальный любящий человек превращаться
__label__0 мама группа выпуск просяк случаться разный ребёнок ребёнок разовый акция время время третий частенько родитель прекрасно курс особенность каждый конкретный ребёнок
__label__1 сука тупой дегенарта видео съести свой старый куколд жухлый сморчок друг друг теребить
__label__1 племя украинец особенно западный детство прививаться мысль самый умный ловко наебал значит молодец понятие подлость честь отсутствовать нацело поэтому маленький дружок весь твой натужный изворотливость работать пообщаться пять минута любой россия понять сорт иметь дело услышать мягкий акцент твой речь
__label__0 пост жадность человек оплатить предоставить халява человек пре

In [ ]:
#help(fasttext.train_supervised)

In [ ]:
classifier = fasttext.train_supervised('train_ft.txt')#, 'model')
result = classifier.test('test_ft.txt')
print('P@1:', result[1])#.precision)
print('R@1:', result[2])#.recall)
print('Number of examples:', result[0])#.nexamples)

P@1: 0.9790914978258858
R@1: 0.9790914978258858
Number of examples: 10809


Read 0M words
Number of words:  26960
Number of labels: 2
Progress: 100.0% words/sec/thread: 1258741 lr:  0.000000 avg.loss:  0.295754 ETA:   0h 0m 0s


In [ ]:
pred = classifier.predict(list(test['lemmas']))[0]
pred = [int(label[0][-1]) for label in pred]

accuracy_score(list(y_test), pred)

0.8717735220649458